# Testing Branch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/pytorch_colab/rolling_and_plot.py .
!cp /content/drive/MyDrive/pytorch_colab/sim_data.csv .

In [ ]:
import numpy as np
import pandas as pd
# !pip install jupyterplot
from jupyterplot import ProgressPlot as PP

from rolling_and_plot import data_plot, normalize, rolling_split, validate
from global_dataclass import G
from battery_lstm import *


%reload_ext autoreload
%autoreload 2

In [ ]:
print(torch.__version__)
print(f"Using {G.device} device")

# TOC

* [Preprocessing](#pre)

* [Data Loading](#dload)

* [Model](#model)

* [Training](#train)

* [Validate](#val)

<a id="pre"></a>
# PreProcessing 

In [ ]:
file = pd.read_csv("/Users/attar/Library/CloudStorage/OneDrive-UniversityofWaterloo/Simulated_Data/sim_data.csv")
# file = pd.read_csv("/content/sim_data.csv")
file["soc"] *= 100 #if sim_data.csv only

In [ ]:
data_plot(data = [file],
          title="OCV v SOC",
          x = ["test time (sec)"],
          y = ["soc"],
          markers = "lines",
          color = "darkorchid",
          x_title = "Test Time (sec)",
          y_title = "SOC"
         )

In [ ]:
file = normalize(file.loc[:,["current","voltage","soc"]].iloc[::G.slicing], G.capacity)

In [ ]:
x_train, x_test, y_train, y_test = rolling_split(file, G.window_size)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

# Data Loader <a id="dload"></a>

In [ ]:
train_dataloader = BatterySet(x_train, y_train)
test_dataloader = BatterySet(x_test, y_test)

train_dataloader = DataLoader(train_dataloader, batch_size=G.batch_size, shuffle=False, drop_last = True)
test_dataloader = DataLoader(test_dataloader, batch_size=G.batch_size, shuffle=False, drop_last = True)

In [ ]:
for X,y in train_dataloader:
    print(f"Shape of X [window, features]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
# for batch, (x,y) in enumerate(test_dataloader.dataset):
#     print(batch,x,y)
#     break

# Model <a id = "model"></a>

In [ ]:
model = LSTMNetwork().to(G.device)
print(model)
# compiled_model = torch.compile(model)

`torch.compile` introduced in the nightly release (makes the model much more efficient). Can use additional arguments to either reduce overhead (and increase memory usage) or make the model as efficient as possible (takes a lot longer to compile).

**Literature**

*Loss*<br>
The LogCoshLoss is the Loss function used by Hannan et al. in their article in the Journal *Nature*: [Deep learning approach towards accurate state of charge estimation for lithium-ion batteries using self-supervised transformer model](https://www.nature.com/articles/s41598-021-98915-8).

*Learning Rate*<br>
The OneCycle learning rate scheduler with cosine annealing introduced by Leslie N. Smith in his paper [A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, momentum, and weight decay](https://doi.org/10.48550/arXiv.1803.09820), seems to be the best scheduler according to Fast.AI

Cosine annealing with warm restarts proposed by Loshchilov et al. in [SGDR: Stochastic Gradient Descent with Warm Restarts](https://doi.org/10.48550/arXiv.1608.03983)

In [ ]:
class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,
                y_pred: torch.Tensor,
                y_true: torch.Tensor) -> torch.Tensor:
        x = torch.sub(y_true, y_pred)
        return torch.sub(
                        torch.add(x, torch.nn.functional.softplus(torch.mul(x, -2.0))),
                        np.log(2.0)
                        )

In [ ]:
loss_fn = LogCoshLoss()

optimizer = torch.optim.Adam(model.parameters(),
                             lr = G.learning_rate,
                             weight_decay= G.weight_decay
                            )

#OneCycle scheduler needs step() to be called after every batch
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                G.learning_rate, #max_lr
                                                epochs = G.epochs,
                                                steps_per_epoch = len(train_dataloader),
                                                anneal_strategy = "cos", #cosine annealing
                                                div_factor = 35,
                                                three_phase = True,
                                                verbose = False
                                                )

#CosineAnnealing with WarmRestarts, step() can be called after every batch but it is dependent of epoch:
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
#                                                                  T_0 = 1,
#                                                                  T_mult = 4,
#                                                                  eta_min = 7e-11,
#                                                                  verbose = False)

In [ ]:
model.load_state_dict(
    torch.load("drive/MyDrive/pytorch_colab/sim_model_state_dict.pth",
    map_location = device)
    )
model.train()

# Training <a id="train"></a>

In [ ]:
pp = PP(plot_names = ["Mean Log Loss", "% Accuracy"],
        line_names = ["Train Loop", "Test Loop"],
        x_label = "epochs"
       )

for epoch in range(1, G.epochs + 1):
    print(f"Epoch {epoch}/{G.epochs}\n----------------------------------------")
    train_loss, train_acc = train_loop(train_dataloader, model, loss_fn, optimizer, scheduler, epoch)
    test_loss, test_acc = test_loop(test_dataloader, model, loss_fn)
    
    pp.update([[train_loss, test_loss], [train_acc, test_acc]])

    # if (epoch != 0) and (epoch % 50 == 0):
    #     torch.save(model.state_dict(), "drive/MyDrive/pytorch_colab/model.pth")
    #     print("Saved the model parameters\n")

print("Completed")

In [ ]:
torch.save(model.state_dict(), "drive/MyDrive/pytorch_colab/sim_model_state_dict.pth")

# Validation <a id="val"></a>

**Dev Set**

In [ ]:
visualize_dev = validate(model, test_dataloader, dev = True)

**Entire Dataset**

In [ ]:
x_set, y_set = rolling_split(file, G.window_size, train = False)

set_dataloader = BatterySet(x_set, y_set)
set_dataloader = DataLoader(set_dataloader, batch_size=G.batch_size, shuffle=False, drop_last = True)

visualize = validate(model, set_dataloader, dev = False)